# Remove Multiplets & low quality cells (based on fraction of reads in peaks, TSS enrichments and clusters that do not annotate to cell type)

We use amulet for multiplet detection. AMULET exploits the expectation that the number of uniquely aligned reads overlapping any given open chromatin region in diploid nuclei ranges from 0 to 2. The method identifies regions with more than two overlapping reads, indicative of multiplets, and uses the Poisson cumulative distribution function to detect significant deviations from expected distributions. The ability to accurately identify and annotate multiplets is crucial for minimizing technical artifacts and ensuring the biological relevance of single-cell genomics findings. After we identify multiplet cells/barcodes, we remove them and reclusters. 

In [ ]:
# Load necessary libraries for data manipulation, visualization, and analysis
library(hdf5r)
library(Seurat)
library(Signac)
library(EnsDb.Hsapiens.v86)
library(dplyr)
library(ggplot2)
install.packages("broom")
if (!requireNamespace('BiocManager', quietly = TRUE))
    install.packages('BiocManager')
BiocManager::install('parallelly')
library(dplyr)
library(ggplot2)
library(sctransform)
library(scater)
library(reticulate)
library(future)
library('Biobase')
library(pheatmap)
library(gplots)
library('hdf5r')
library(EnsDb.Hsapiens.v86)
library(BiocParallel)
library(tictoc)
library(Seurat)
library(Signac)
library(EnsDb.Hsapiens.v86)
library(ggplot2)
library(cowplot)
library(GenomeInfoDb)

suppressMessages(library(hdf5r))
suppressMessages(library(Seurat))
suppressMessages(library(Signac))
suppressMessages(library(EnsDb.Hsapiens.v86))
suppressMessages(library(dplyr))
suppressMessages(library(ggplot2))
suppressMessages(library(Matrix))
suppressMessages(library(harmony))
suppressMessages(library(data.table))
suppressMessages(library(ggpubr))
suppressMessages(library(future))

.libPaths()
BiocManager::install("Bioconductor/GenomeInfoDb",lib = "/home/parulk/R/x86_64-pc-linux-gnu-library/4.1",force = TRUE)
library(GenomeInfoDb,lib.loc="/home/parulk/R/x86_64-pc-linux-gnu-library/4.1")
packageVersion("GenomeInfoDb",lib.loc="/home/parulk/R/x86_64-pc-linux-gnu-library/4.1")
GenomeInfoDb::getChromInfoFromUCSC("hg38")

# Multiplet Detection

We use amulet for multiplet detection. AMULET exploits the expectation that the number of uniquely aligned reads overlapping any given open chromatin region in diploid nuclei ranges from 0 to 2. The method identifies regions with more than two overlapping reads, indicative of multiplets, and uses the Poisson cumulative distribution function to detect significant deviations from expected distributions. The ability to accurately identify and annotate multiplets is crucial for minimizing technical artifacts and ensuring the biological relevance of single-cell genomics findings.

### Run AMULET 

<code>{path_to_amulet_installation}/AMULET.sh --forcesorted --bambc CB --bcidx 0 --cellidx 8 --iscellidx 9 {path_to_cellranger_output}/${sample}//outs/possorted_bam.bam {path_to_cellranger_output}/${sample}/outs/singlecell.csv {path_to_amulet_installation}/human_autosomes.txt /RepeatFilterFiles/blacklist_repeats_segdups_rmsk_hg38.bed {path_to_amulet_output}/HPAP-040 {/path/to/shellscript/}</code>

### Amulet Output
The multiplet detection python script produces three output files: MultipletProbabilities, MultipletCellIds_xx.txt, and MultipletBarcodes_xx.txt (xx corresponding to the q-value threshold used to call multiplets).

#### MultipletProbabilities.txt
A tab delimited file with the following columns:

1. cell_id - The cell id (e.g., _cell_0, _cell_1, etc. from CellRanger)
2. barcode - The cell barcode.
3. p-value - The Poisson probability obtained from the cumulative distribution function.
4. q-value - The FDR corrected p-value for multiple hypothesis testing.

#### MultipletCellIds_xx.txt
Files with the MultipletCellIds prefix correspond to multiplet cell ids with a q-value threshold specified by xx (i.e., 0.xx). For example 01 implies a q-value threshold of 0.01.

#### MultipletBarcodes_xx.txt
Files with the MultipletBarcodes prefix correspond to multiplet cell barcodes with a q-value threshold specified by xx (i.e., 0.xx). For example 01 implies a q-value threshold of 0.01.

#### Content of samples.txt
HPAP-035 HPAP-036 HPAP-039 HPAP-040 HPAP-044 HPAP-045 HPAP-047 HPAP-049 HPAP-050 HPAP-051 HPAP-052 HPAP-053 HPAP-054 HPAP-055 HPAP-056 HPAP-059 HPAP-061 HPAP-062 HPAP-063 HPAP-064 HPAP-067 HPAP-069 HPAP-072 HPAP-075 HPAP-077 HPAP-079 HPAP-080 HPAP-081 HPAP-083 HPAP-084 HPAP-085 HPAP-088 HPAP-092 HPAP-099 HPAP-100 HPAP-101 HPAP-103 HPAP-104 HPAP-105 HPAP-106 HPAP-109

In [ ]:
# the code reads in barcodes from the "MultipletBarcodes_01.txt" file for each sample listed in "SAMPLES.txt" and stores them in the multiplets list.
samples<- scan('/HPAP_scATAC/samples.txt', what='', sep=' ')
multiplets<-list()
#metrics<- list()
for (samp in samples){
    multiplets[[samp]] <- read.table(sprintf("{path_to_amulet_output}/%s/MultipletBarcodes_01.txt", samp)) %>% t() %>% as.vector()
}

head(multiplets[[length(multiplets)]])

In [ ]:
#read in RDS file that is output of 01_Seurat_snATAC_windows_Harmony_reducePCs.ipynb
adata<- readRDS(file = '/HPAP_scATAC/HVW_all_samples_harmony_reduced_final_gene_activity.rds')


In [ ]:
options(repr.plot.width=10, repr.plot.height=10)
DimPlot(object = adata, label = TRUE) + NoLegend()


In [ ]:
cells<- Cells(adata)

In [ ]:
# Process barcode values for each sample by concatenating them with the sample name
new_b<- list()
for (samp in samples){
    barcode<-multiplets[[samp]]
    for (i in 1:length(barcode)){
        new_b[[samp]][i]<-paste(samp,barcode[i], sep='_')
    }
}

In [ ]:
length(new_b[[samp]])
length(cells)

In [ ]:
# Initialize multiplet.metadata list to classify cells as singlet or multiplet based on barcodes
multiplet.metadata <-list()

for (samp in samples){
    multiplet.metadata[[samp]]<- rep("singlet", length(cells))
    names(multiplet.metadata[[samp]])<- cells
    vec<- multiplet.metadata[[samp]]
    mult<-new_b[[samp]]
    for (i in 1:length(names(vec))){
    if (names(vec[i]) %in% mult){
        vec[i]<- "multiplet"
}
        }
    multiplet.metadata[[samp]]<-vec
    }

In [ ]:
# Sanity check to ensure correct barcode processing
for (i in 1:length(new_b)){
    if (i == 1){
        mult<- new_b[[i]]
    } else {
        mult<- append(mult, new_b[[i]])
    }
}
if (length(mult) == sum(unlist(lapply(multiplets, length)))){
    message('Done.')
    } else {
    message('An error has occurred')
}

In [ ]:
length(mult)

# Remove Doublets

In [ ]:
#remove multiplet cells from amulet output
init<- length(Cells(adata))
adata$remove_cells <-(Cells(adata) %in% mult)

DimPlot(adata, group.by = "remove_cells", pt.size = 0.5, 
        cols = c("FALSE" = "grey", "TRUE" = "purple"), 
        order = c("TRUE", "FALSE"))

true_count <- sum(adata$remove_cells == TRUE)
false_count <- sum(adata$remove_cells == FALSE)
total <- true_count + false_count
true_percent <- (true_count / total) * 100
false_percent <- (false_count / total) * 100

# Create a pie chart
pie(c(true_count, false_count),  col = c("purple", "grey"))

# Add a legend
legend("topright", c(paste("Multiplet",true_percent, "%"), paste("Singlet", false_percent, "%")), fill = c("purple", "grey"))

count <- sum(adata$remove_cells == TRUE)
print(count)

In [ ]:
# Remove the multiplet cells from the Seurat object and perform the downstream analyses.
sub_adata<- subset(adata, subset=remove_cells==FALSE)
final<- length(Cells(sub_adata))
    if (final/init < 1){
        message('Completed doublet removal')
    } else {
        message('Some error has occured')
    }

In [ ]:
options(repr.plot.width=10, repr.plot.height=10)
DimPlot(object = sub_adata, label = TRUE) + NoLegend()

# After removal of multiplet cells, recluster i.e. dimensionality reduction via Harmony, and clustering

In [ ]:
DefaultAssay(sub_adata) <- 'ATAC_windows'

sub_adata <- RunTFIDF(sub_adata)
sub_adata <- FindTopFeatures(sub_adata, min.cutoff='q0', verbose=FALSE)

sub_adata <- RunSVD(sub_adata)

DepthCor(sub_adata)

sub_adata@reductions

hm_atac <- HarmonyMatrix(Embeddings(sub_adata, reduction='lsi'),sub_adata@meta.data,  c("library","sex"), do_pca=FALSE,plot_convergence = TRUE, verbose = TRUE)


sub_adata[['harmony.atac']] <- CreateDimReducObject(embeddings=hm_atac, key='LSI_', assay= 'ATAC_windows')

sub_adata <- RunUMAP(sub_adata, dims=2:30, reduction='harmony.atac', reduction.name='umap.atac', reduction.key='atacUMAP_')


options(repr.plot.width=10, repr.plot.height=10)
p3 <- DimPlot(sub_adata, reduction='umap.atac', group.by = 'library', label=TRUE, label.size=6, repel=TRUE, raster=FALSE) + ggtitle('WNN')
p3 <- p3 + xlab('UMAP 1') + ylab('UMAP 2') + ggtitle('ATAC')
p3

sub_adata@reductions

DepthCor(sub_adata, reduction = 'harmony.atac')

sub_adata <- FindNeighbors(object = sub_adata, reduction = 'harmony.atac', dims = 2:30)
sub_adata <- FindClusters(object = sub_adata, algorithm=4,resolution = 1.5,method = "igraph") 

DimPlot(object = sub_adata, label = TRUE) + NoLegend()


In [ ]:
options(repr.plot.width=10, repr.plot.height=10)
DimPlot(object = sub_adata, group.by = 'library',label = FALSE)# + NoLegend()
DimPlot(object = sub_adata, group.by = 'condition',label = FALSE)# + NoLegend()

# Plot  quality control metrics such as fraction of reads in peaks, fraction of reads in promoter, TSS enrichment and nuclosome signal to determine cutoff to remove low quality cells

In [ ]:
#add qc metrics
samples <- c('HPAP-035', 'HPAP-036', 'HPAP-039', 'HPAP-040', 'HPAP-044', 'HPAP-045', 'HPAP-047', 'HPAP-049', 'HPAP-050', 'HPAP-051', 'HPAP-052', 'HPAP-053', 'HPAP-054', 'HPAP-055', 'HPAP-056', 'HPAP-059', 'HPAP-061', 'HPAP-062', 'HPAP-063', 'HPAP-064', 'HPAP-067', 'HPAP-069', 'HPAP-072', 'HPAP-075', 'HPAP-077', 'HPAP-079', 'HPAP-080', 'HPAP-081', 'HPAP-083', 'HPAP-084', 'HPAP-085', 'HPAP-088', 'HPAP-092', 'HPAP-099', 'HPAP-100', 'HPAP-101', 'HPAP-103', 'HPAP-104', 'HPAP-105', 'HPAP-106', 'HPAP-109')
qcs <- list()
for (sample in samples) {
    wd <- sprintf('/HPAP_scATAC/lfm/')
    qc <- read.table(file.path(wd, sprintf('%s.qc_metrics.txt', sample, sample)), sep='\t', header=TRUE, stringsAsFactors=FALSE)
    qcs[[sample]] <- qc
}
qc <- as.data.frame(rbindlist(qcs))
qc$X <- paste0(qc$X, "-1")

head(qc)
rownames(qc) <- qc$X
qc <- qc[Cells(sub_adata), 6:length(colnames(qc))]
sub_adata <- AddMetaData(object=sub_adata, metadata=qc)
qc <- qcs <- NULL
gc()
head(sub_adata)
metadata <- sub_adata@meta.data
head(sub_adata@meta.data)

In [ ]:
options(repr.plot.width=20, repr.plot.height=20)

p1 <- VlnPlot(sub_adata, features='TSS.enrichment',  pt.size=0, log = TRUE, split.by = 'seurat_clusters' ) + geom_boxplot(width=.6, fill='white', alpha=.6) + geom_hline(yintercept=median(sub_adata$TSS.enrichment), linetype='dashed')#
p2 <- VlnPlot(sub_adata, features='nucleosome_signal',  pt.size=0, log = TRUE, split.by = 'seurat_clusters' ) + geom_boxplot(width=.6, fill='white', alpha=.6) + geom_hline(yintercept=median(sub_adata$nucleosome_signal), linetype='dashed')#
p3 <- VlnPlot(sub_adata, features='frac_reads_in_peaks',  pt.size=0,  split.by = 'seurat_clusters') + geom_boxplot(width=.6, fill='white', alpha=.6) + geom_hline(yintercept=median(sub_adata$frac_reads_in_peaks), linetype='dashed')#
p4 <- VlnPlot(sub_adata, features='frac_reads_in_promoters',  pt.size=0, split.by = 'seurat_clusters') + geom_boxplot(width=.6, fill='white', alpha=.6) + geom_hline(yintercept=median(sub_adata$frac_reads_in_promoters), linetype='dashed')#

p1 / p2 / p3 / p4

In [ ]:
table(Idents(sub_adata))

In [ ]:
metadata <- sub_adata@meta.data
frac_reads_in_peaks <- sub_adata@meta.data$frac_reads_in_peaks
quantile(frac_reads_in_peaks)
ggplot(data=metadata, mapping = aes(x=frac_reads_in_peaks)) +  geom_density(alpha = 0.2, color="green", fill="lightgreen") + 
theme_linedraw() + geom_vline(xintercept=c(0.33,0.6,0.7), colour=c("blue", "red", "black"),linetype = "longdash")

In [ ]:
frac_reads_in_promoters <- sub_adata@meta.data$frac_reads_in_promoters
quantile(frac_reads_in_promoters)
ggplot(data=metadata, mapping = aes(x=frac_reads_in_promoters)) +  geom_density(alpha = 0.2, fill= 'lightpink', color="pink") + 
theme_linedraw() + geom_vline(xintercept=c(0.03,0.038,0.041), colour=c("blue", "red", "black"),linetype = "longdash")

In [ ]:
TSS.enrichment <- sub_adata@meta.data$TSS.enrichment
quantile(TSS.enrichment)
ggplot(data=metadata, mapping = aes(x=TSS.enrichment)) +  geom_density(alpha = 0.2, color="blue", fill="lightblue") + 
theme_linedraw() + geom_vline(xintercept=c(4.24,4.79,5.39), colour=c("blue", "red", "black"),linetype = "longdash")
sub_adata$high.tss <- ifelse(sub_adata$TSS.enrichment > 2, 'High', 'Low')
TSSPlot(sub_adata, group.by = 'high.tss') + NoLegend()

In [ ]:
nucleosome_signal <- sub_adata@meta.data$nucleosome_signal

quantile(nucleosome_signal)

ggplot(data=metadata, mapping = aes(x=nucleosome_signal)) +  geom_density(alpha = 0.2,color="yellow", fill="yellow") + 
theme_linedraw() 
sub_adata$nucleosome_group <- ifelse(sub_adata$nucleosome_signal > 4, 'NS > 4', 'NS < 4')
FragmentHistogram(object = sub_adata, group.by = 'nucleosome_group')

# Marker Gene plot to assign cell type to cluster 

In [ ]:
library(ggpubr)
library(ggbreak)
library(gridExtra)
library(grid)
library(ggh4x)
library(ggplot2)
library(ggforce)
library('tidyr')

# Load markers list
project.dir = "/HPAP_scATAC/"
cell.markers = read.table("/HPAP_scATAC/Cell.markers.txt", sep = ',', header = TRUE)
# Make it long, remove useless column and void markers
cell.markers <- cell.markers %>% gather(Key, marker, c(3:ncol(cell.markers)))
cell.markers = cell.markers[,-3]
cell.markers = cell.markers[cell.markers$marker != "", ]
head(cell.markers)
# Factorize columns
cell.markersCompartment <- cell.markers$cell.markersCompartment
cell.markersCellType <- cell.markers$cell.markersCellType
cell.markersCompartment = factor(cell.markersCompartment, levels = c("Endocrine cells", "Non-endocrine cells"))
cell.markersCellType = factor(cell.markersCellType, levels = c("Beta", "Alpha", "Delta", "Gamma", "Epsilon", "Ductal", "MUC5B Ductal", "Acinar", "Stellate", "Act. Stellate", "Q. Stellate", "Endothelial", "T Cell", "Schwann", "Macrophages", "Dividing Cells"))

g = DotPlot(sub_adata, assay='RNA', features=cell.markers$marker, cluster.idents=TRUE, col.min=0) +
        theme(axis.text.x=element_text(angle=45, hjust=1)) + xlab('') + ylab('')
    meta_summary = g$data
    colnames(meta_summary)[3] = "marker"
    meta_summary = merge(meta_summary, cell.markers, by = "marker")

    options(repr.plot.width=25, repr.plot.height=10)
    figure <- ggplot(meta_summary, aes(x = marker, y = id)) +
      geom_point(aes(size = pct.exp, fill = avg.exp.scaled, stroke=NA),
                 shape = 21) +
      scale_size("% detected", range = c(0, 6)) +
      scale_fill_gradient(low = "lightgray", high = "blue",
                           guide = guide_colorbar(nbin = 200,
                                                  ticks.colour = "black", frame.colour = "black"),
                           name = "Average\nexpression") +
      ylab("Cluster") + xlab("") +
      theme_bw() +
      theme(axis.text = element_text(size = 100),
            axis.text.x = element_text(size = 10, angle = 45, hjust = 1, color = "black"),
            strip.text.x = element_text(size = 10),
            axis.text.y = element_text(size = 12, color = "black"),
            axis.title = element_text(size = 14)) +
facet_nested(~ Compartment + CellType, scales = "free")

figure

In [ ]:
covariant.ls = c("library", "sex", "condition")

gg.ls = list()

# Library
    i = 1
    covariant = covariant.ls[i]
    Covariant.table = as.data.frame(table(sub_adata$seurat_clusters, sub_adata$library))
    colnames(Covariant.table) = c("cluster", "covariant", "Freq")
    gg.ls[[i]] = ggplot(Covariant.table, aes(fill = covariant, y = Freq, x = cluster)) +
      theme_bw() +
      coord_flip() +
      geom_bar(position = position_fill(reverse = TRUE), stat = 'identity', color = 'black', size = 0.2) +
      labs(y= "\n Percentage", x = "", title = covariant) + 
      theme(axis.text = element_text(size = 12), axis.title = element_text(size = 12, face = "bold"),
                      axis.text.x = element_text(angle = 90),
                                 plot.title = element_text(size = 18, face = "bold", , hjust = 0.5))

# Sex
    i = 2
    covariant = covariant.ls[i]
    Covariant.table = as.data.frame(table(sub_adata$seurat_clusters, sub_adata$sex))
    colnames(Covariant.table) = c("cluster", "covariant", "Freq")
    gg.ls[[i]] = ggplot(Covariant.table, aes(fill = covariant, y = Freq, x = cluster)) +
      theme_bw() +
      coord_flip() +
      geom_bar(position = position_fill(reverse = TRUE), stat = 'identity', color = 'black', size = 0.2) +
      labs(y= "\n Percentage", x = "", title = covariant) + 
      theme(axis.text = element_text(size = 12), axis.title = element_text(size = 12, face = "bold"),
                      axis.text.x = element_text(angle = 90),
                                 plot.title = element_text(size = 18, face = "bold", , hjust = 0.5))

# Condition
    i = 3
    covariant = covariant.ls[i]
    Covariant.table = as.data.frame(table(sub_adata$seurat_clusters, sub_adata$condition))
    colnames(Covariant.table) = c("cluster", "covariant", "Freq")
    gg.ls[[i]] = ggplot(Covariant.table, aes(fill = covariant, y = Freq, x = cluster)) +
      theme_bw() +
      coord_flip() +
      geom_bar(position = position_fill(reverse = TRUE), stat = 'identity', color = 'black', size = 0.2) +
      labs(y= "\n Percentage", x = "", title = covariant) + 
      theme(axis.text = element_text(size = 12), axis.title = element_text(size = 12, face = "bold"),
                      axis.text.x = element_text(angle = 90),
                                 plot.title = element_text(size = 18, face = "bold", , hjust = 0.5))

options(repr.plot.height = 20, repr.plot.width = 40)
cp <- gg.ls[[1]] + gg.ls[[2]] + gg.ls[[3]]
cp

# Remove low quality cells based on fraction of reads in peaks, TSS enrichment &  clusters that do not annotate to cell type

In [ ]:
# Use quality control metrics such as fraction of reads in peaks, TSS enrichment and remove clusters that do not annotate to a cell type (based on marker gene plot)
sub_adata <- subset(x=sub_adata, subset = frac_reads_in_peaks > 0.3)

sub_adata <- subset(x=sub_adata, subset = TSS.enrichment > 3)

sub_adata <- subset(x = sub_adata, idents = c(5, 18, 20, 22, 24, 25, 26), invert = TRUE)

table(sub_adata[[]]$seurat_clusters)

# After removal of low quality cells, recluster i.e. dimensionality reduction via Harmony, and clustering

In [ ]:
DefaultAssay(sub_adata) <- 'ATAC_windows'
sub_adata <- RunTFIDF(sub_adata)
sub_adata <- FindTopFeatures(sub_adata, min.cutoff='q0', verbose=FALSE)

sub_adata <- RunSVD(sub_adata)

DepthCor(sub_adata)

sub_adata@reductions

hm_atac <- HarmonyMatrix(Embeddings(sub_adata, reduction='lsi'),sub_adata@meta.data,  c("library","sex"), do_pca=FALSE,plot_convergence = TRUE, verbose = TRUE)


sub_adata[['harmony.atac']] <- CreateDimReducObject(embeddings=hm_atac, key='LSI_', assay= 'ATAC_windows')

sub_adata <- RunUMAP(sub_adata, dims=2:30, reduction='harmony.atac', reduction.name='umap.atac', reduction.key='atacUMAP_')


options(repr.plot.width=10, repr.plot.height=10)
p3 <- DimPlot(sub_adata, reduction='umap.atac', group.by = 'library', label=TRUE, label.size=6, repel=TRUE, raster=FALSE) + ggtitle('WNN')
p3 <- p3 + xlab('UMAP 1') + ylab('UMAP 2') + ggtitle('ATAC')
p3

sub_adata@reductions

DepthCor(sub_adata, reduction = 'harmony.atac')

In [ ]:
sub_adata <- FindNeighbors(object = sub_adata, reduction = 'harmony.atac', dims = 2:30)
sub_adata <- FindClusters(object = sub_adata, algorithm=4,resolution = 1.5,method = "igraph") 

DimPlot(object = sub_adata, label = TRUE) + NoLegend()


In [ ]:
saveRDS(sub_adata, '/HPAP_scATAC/HVW_all_samples_harmony_reduced_final_gene_activity_amulet_reclustered_final.rds')

In [ ]:
sub_adata

In [ ]:
libraries <- sub_adata$library

In [ ]:
head(libraries)

In [ ]:
write.csv(libraries, '/HPAP_scATAC/barcodes_windows_cluster.csv')